In [7]:
import pandas as pd
import feather
import numpy as np

In [9]:
data = pd.read_csv('train.csv', index_col=0)
target = 'target'
data.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [12]:
test_data = pd.read_csv('test.csv', index_col=0)
test_data.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [14]:
from tqdm import tqdm

# Feature selection:
# We will make relationships based on values instead of columns (because there's a whole thread
# in kaggle explaining there's no relationship between them but there actually are for same target
# values)
# Eg. targets == 0 may have similarities in it's features
# We will try to look for:
# Values that appear at least another time in data with target==1 and no 0;
# Values that appear at least another time in data with target==0 and no 1;
# Values that appear at least two more times in data with target==0 & 1;
# Values that appear only once (unique values)
original = [f'var_{i}' for i in range(200)]
has_one = [f'var_{i}_has_one' for i in range(200)]
has_zero = [f'var_{i}_has_zero' for i in range(200)]
not_u = [f'var_{i}_not_unique' for i in range(200)]

for f in tqdm(original):
    unique_v = test_data[f].value_counts()
    unique_v = unique_v.index[unique_v == 1]
    test_data[f + '_u'] = test_data[f].isin(unique_v)

100%|██████████| 200/200 [00:03<00:00, 52.23it/s]


In [17]:
# Find unique values, save it
test_data['has_unique'] = test_data[[f + '_u' for f in original]].any(axis=1)
print(test_data['has_unique'].sum())

100000


In [18]:
# Find the data that matched, concatenate to the real samples (so we keep the feature)
real_samples = test_data.loc[test_data['has_unique'], original]
ref = pd.concat([data, real_samples], axis=0)
print(ref.shape)

(300000, 201)


In [19]:
# Perform the same feature selection but for the others (has_one/has_zero -- based on target equality (==0||1))
# This is done on the DATA set
for f in tqdm(original):
    data[f + '_has_one'] = 0
    data[f + '_has_zero'] = 0
    f_1 = data.loc[data[target] == 1, f].value_counts()
    
    f_1_1 = set(f_1.index[f_1 > 1])
    f_0_1 = set(f_1.index[f_1 > 0])

    f_0 = data.loc[data[target] == 0, f].value_counts()
    f_0_0 = set(f_0.index[f_0 > 1])
    f_1_0 = set(f_0.index[f_0 > 0])
    
    data.loc[data[target] == 1, f + '_has_one'] = data.loc[data[target] == 1, f].isin(f_1_1).astype(int)
    data.loc[data[target] == 0, f + '_has_one'] = data.loc[data[target] == 0, f].isin(f_0_1).astype(int)

    data.loc[data[target] == 1, f + '_has_zero'] = data.loc[data[target] == 1, f].isin(f_1_0).astype(int)
    data.loc[data[target] == 0, f + '_has_zero'] = data.loc[data[target] == 0, f].isin(f_0_0).astype(int)

data.loc[:, has_one] = 2*data.loc[:, has_one].values + data.loc[:, has_zero].values

100%|██████████| 200/200 [01:43<00:00,  1.17it/s]


In [20]:
# Apply over the test_data the feature selection above
for f in tqdm(original):
    test_data[f + '_has_one'] = 0
    test_data[f + '_has_zero'] = 0
    f_1 = data.loc[data[target] == 1, f].unique()
    f_0 = data.loc[data[target] == 0, f].unique()
    test_data.loc[:, f + '_has_one'] = test_data[f].isin(f_1).astype(int)
    test_data.loc[:, f + '_has_zero'] = test_data[f].isin(f_0).astype(int)
    
test_data.loc[:, has_one] = 2*test_data.loc[:, has_one].values + test_data.loc[:, has_zero].values

100%|██████████| 200/200 [01:21<00:00,  1.27it/s]


In [21]:
# For each col, add non_unique fields (to data and test_data set)
for f in tqdm(original):
    v = ref[f].value_counts()
    
    non_unique_v = v.index[v != 1]
    
    m_data = data[f].isin(non_unique_v)
    data[f + '_not_unique'] = m_data  * data[f] + (~m_data) * data[f].mean()
    
    m_test_data = test_data[f].isin(non_unique_v)
    test_data[f + '_not_unique'] = m_test_data  * test_data[f] + (~m_test_data) * test_data[f].mean()
    
    data.loc[~m_data, f + '_has_one'] = 4
    test_data.loc[~m_test_data, f + '_has_one'] = 4

100%|██████████| 200/200 [04:00<00:00,  1.51s/it]


In [22]:
# Count value matches for var_0 column
data['var_0_has_one'].value_counts()

1    126381
3     27409
4     23064
0     17291
2      5855
Name: var_0_has_one, dtype: int64

In [24]:
# Save feature-selection feathers
feather.write_dataframe(data.reset_index(), './data_feather.fth')
feather.write_dataframe(test_data.reset_index(), './test_data_feather.fth')
np.save('./real_samples.index', real_samples.index.values)